<a href="https://colab.research.google.com/github/vinay10949/AnalyticsAndML/blob/master/Python/Catboost_model_export_as_json_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# CatBoost JSON model tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/catboost/tutorials/blob/master/model_analysis/model_export_as_json_tutorial.ipynb)

CatBoost supports exporting model to JSON format and loading model from it.

This tutorial explains the structure of the JSON model with numeric features only.

Download MSRank dataset

In [0]:
import catboost
from catboost import datasets
import os
import numpy as np

train_df, _ = datasets.msrank_10k()
X, Y = train_df[train_df.columns[1:]], train_df[train_df.columns[0]]
pool = catboost.Pool(
    data=X[:1000], # top 1000 documents are enough for this example
    label=Y[:1000],
    feature_names=list(X.columns)
)

Now we will train a simple model with trees of depth 2

In [0]:
cls = catboost.CatBoostClassifier(depth=2, random_seed=0, iterations=10, verbose=False)
cls.fit(pool)
approx = cls.predict(X[0:3], prediction_type="RawFormulaVal")

The next block save JSON model to file.

In [0]:
cls.save_model(
    "model.json",
    format="json",
    # pool=pool  # this parameter is required only for models with categorical features.
)

The next block loads model from file as JSON and shows its keys.
The model json contains __model_info__, __oblivious_trees__ and __features_info__. Model with categorical features will also conatain __ctrs__.

In [0]:
import json
model = json.load(open("model.json", "r"))
model.keys()

[u'model_info', u'oblivious_trees', u'features_info']

### Model info
model['model_info'] is analogue for [get_metadata()](https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_catboost_metadata-docpage/) function.
You can look on the training parameters the model was trained with or on catboost version that the model was trained with.

In [0]:
print(model['model_info']['catboost_version_info'])

Svn info:
    URL: svn+ssh://arcadia.yandex.ru/arc/trunk/arcadia
    Last Changed Rev: 5437739
    Last Changed Author: dkvasov
    Last Changed Date: 2019-08-08T13:43:02.471284Z

Other info:
    Build by: eermishkina
    Top src dir: /place/home/eermishkina/trunc/arcadia
    Top build dir: /home/eermishkina/.ya/build
    Hostname: su57.search.yandex.net
    Host information: 
        Linux su57.search.yandex.net 4.4.88-42 #1 SMP Mon Sep 18 14:33:37 UTC 2017 x86_64

    


### Features info

Let's look now on the features_info value.
It could contain $float\_features$, $categorical\_features$ and $ctrs$, which are lists of descriptions of some features.

In [0]:
model['features_info'].keys()

[u'float_features']

In our case (model without cateforical features) there are only one fields - $float\_features$.


<a id='features_info'></a>
Every float feature is described in the following way:
__flat_feature_index__ (int) - feature index in pool, zero-based indexation 

__feature_index__ (int) index among only float features, zero-based indexation. For example, in dataset that looks like \[float, categ, float\], the second float feature has indices $flat\_feature\_index = 2$ and $feature\_index = 1$. 
Because it is the 2 feature of all in 0 based indexation and 1 feature of numeric ones (here we exclude the caterorical feature).
For a model without categorical features $feature\_index$ will be equal to $float\_feature\_index$ for every feature.

__borders__ (list of all borders (or splits) used in the model for this particular float feature). Float feature values can be splits by $border$ value. All elements with feature value $> border$ go to the left subtree, and all elements with feature value ($<= border$) elements go to the right subtree.

__has_nans__ (bool) This field shows if there were any nan values in the training dataset, which was used to train the model.

__nan_value_treatment__ ('AsIs', 'AsTrue' or 'AsFalse') If the feature has had nan values in the training dataset, then there is an additional split that puts nans to the left and everything else to the right (if 'AsFalse') or an additional split that puts nans to the right and everything else to the left (if 'AsTrue').
'AsIs' is internal default value for features without nan values.

In [0]:
model['features_info']['float_features'][0]

{u'borders': [83.5],
 u'feature_index': 0,
 u'flat_feature_index': 0,
 u'has_nans': False,
 u'nan_value_treatment': u'AsIs'}

This shows that the first feature has a list of borders that were used in the model. And this feature had no nan values in train.

### Symmetric trees

CatBoost uses so-called symmetric or oblivious trees. For each level of the tree CatBoost uses the same features to split learning instances into the left and the right partitions: on the first level tree is partitioned by first split into two parts, on the second level each subtree splits with second split and so on.

In this case a tree of depth $k$ has exactly $2^k$ leaves and $k$  splits, each split on a subsequent layer.

There are three types of splits: "FloatFeature", "OneHotFeature" and "OnlineCtr". A model without cateforical features contains only float feature slits.

Now, let's look on how JSON model describes a single tree.
A tree of depth $k$ is described by $2^k$ leaf_values, $2^k$ leaf_weights and $k$ splits. Let's take a look on the first tree of our model.

In [0]:
def dump_json(item):
    print(json.dumps(item, indent=2))

dump_json(model['oblivious_trees'][0])  # first_tree

{
  "leaf_values": [
    0.022173912547853145, 
    0.017826086558078085, 
    0.011304347573415556, 
    -0.02565217333967221, 
    -0.02565217333967221, 
    0.07652173742004059, 
    -0.016956521360122625, 
    -0.019130434355010134, 
    -0.02130434734989765, 
    -0.019130434355010148, 
    0.02407969585645712, 
    0.027495255552409406, 
    0.0035863376807432745, 
    -0.026869069608165402, 
    -0.028292219481478875, 
    0.07398772840759553, 
    -0.004233128739738201, 
    -0.012975459832675437, 
    -0.0286196312621424, 
    -0.02815950857304045
  ], 
  "splits": [
    {
      "split_index": 4, 
      "float_feature_index": 16, 
      "border": 10.407758712768555, 
      "split_type": "FloatFeature"
    }, 
    {
      "split_index": 3, 
      "float_feature_index": 13, 
      "border": 3.5, 
      "split_type": "FloatFeature"
    }
  ], 
  "leaf_weights": [
    123, 
    54, 
    512, 
    311
  ]
}


The list of "leaf_values" describes the values in leaves. This is a tree with 4 leaves. It has depth 2, which means it has two different splits. The first split is used to split all the objects into left and right.
And the second split is used twice, to split the left objects into two parts, and to split the right objects into two parts.
The indices in the list can be represented using base-2 numeral system in the following way: 00, 01, 10, 11. Leaf 00 is the leaf where the 0-th split and the 1-st split are equal to False. Leaf 01 contains the objects where 0-th split is equal to False and 1-st feature is equal to True. And so on.

The next part of the tree description is called "leaf_weights". This list represents sum of weights of training samples, that are in this leaf. Leaf indexation in this list is the same as in "leaf_values".

The last part is "splits", and it is description of the two splits that are used in the tree of depth two.
Each of the descriptions contains several key-values. Firstly, it contains internal CatBoost parameter $split\_index$. This is the only parameter that is used by catboost when loading the model, all other parts of in "splits" are ignored (they are duplicated in a different place), and are present here only to do the model easier to understand.


Let's first describe the other fields. Split type "FloatFeature" means that it is so called 'float split'. Float split condition $float\_feature[float\_feature\_index] > border$ (see above in [Features info](#features_info)) is decripted with "float_feature_index" and "border" accordingly.


This description should enable you to analyze the model.
But if you will want to change the model, you will have to change "split_index" in a right way.
To to do that let's explain, how this feature is built. Look one more time at features info:


In [0]:
feature = model['features_info']['float_features'][0]
feature

{u'borders': [83.5],
 u'feature_index': 0,
 u'flat_feature_index': 0,
 u'has_nans': False,
 u'nan_value_treatment': u'AsIs'}

Each float split determines by feature index and border value, hence feature description specifies len(feature['borders']) splits. List all float splits with first feature from model['features_info'] with border value from features $borders$ list, with second feature and so on. This is the order, in which splits are enumerated in  model. Splits numbering begins with 0. Build split list in this order


In [0]:
split_list = []
for float_feature in model['features_info']['float_features']:
    if not float_feature['borders']:
        continue
    for border in float_feature['borders']:
        split_list.append(
            {
                'split_index': len(split_list),
                'float_feature_index': float_feature['feature_index'], 
                'border_id': border, 
                'split_type': 'FloatFeature',
                'flat_feature_index': float_feature['flat_feature_index']
            }
        )

Ensure, that splits in first tree and corresponding splits from obtained above split_list are identical

In [0]:
first_tree = model['oblivious_trees'][0]
first_tree['splits']

[{u'border': 10.407758712768555,
  u'float_feature_index': 16,
  u'split_index': 4,
  u'split_type': u'FloatFeature'},
 {u'border': 3.5,
  u'float_feature_index': 13,
  u'split_index': 3,
  u'split_type': u'FloatFeature'}]

In [0]:
split_indexes = [x['split_index'] for x in first_tree['splits']]
[split_list[index] for index in split_indexes]

[{'border_id': 10.407758712768555,
  'flat_feature_index': 16,
  'float_feature_index': 16,
  'split_index': 4,
  'split_type': 'FloatFeature'},
 {'border_id': 3.5,
  'flat_feature_index': 13,
  'float_feature_index': 13,
  'split_index': 3,
  'split_type': 'FloatFeature'}]

## Multiclassification

The only difference between classification or regression vs multiclassification is the leaves count in each tree. Model contains leaf values for each class, so tree depth of $k$ has $2^k$ leafs and in json model are stored  $2^k \cdot classes\_count$ leaf values in this order: first $2^k$ values for first class, second  $2^k$ values for second and so on. Leaf weights count is $2^k$ as they are the same for all classes. Look at first tree of multiclass model trained on  [Iris](https://en.wikipedia.org/wiki/Iris_flower_data_set) dataset.

In [0]:
# Get Iris dataset 
from sklearn import datasets
iris = datasets.load_iris()

# Train the model
cls_multilclass = catboost.CatBoostClassifier(loss_function='MultiClass', depth=2, random_seed=0, verbose=False)
cls_multilclass.fit(iris.data, iris.target)

# Save model
cls_multilclass.save_model(
    "multiclass_model.json",
    format="json",
    # pool=pool  # is required for model with cat_features to obtain applicable model
)

multilclass_model = json.load(open("multiclass_model.json", "r"))
multilclass_model['oblivious_trees'][0]

{u'leaf_values': [0.05084745649058943,
  -0.025423728245294732,
  -0.025423728245294732,
  -0.02526315733006127,
  0.04578947266073611,
  -0.020526315330674765,
  0,
  0,
  0,
  -0.025573769920184966,
  -0.018196720904746992,
  0.04377049082493207],
 u'leaf_weights': [50, 48, 0, 52],
 u'splits': [{u'border': 0.800000011920929,
   u'float_feature_index': 3,
   u'split_index': 58,
   u'split_type': u'FloatFeature'},
  {u'border': 1.5499999523162842,
   u'float_feature_index': 3,
   u'split_index': 64,
   u'split_type': u'FloatFeature'}]}

### Truncate model
Model can be modified and applied. Truncate and apply model

In [0]:
trees = model['oblivious_trees'][:]

In [0]:
approx # = cls.predict(X[0:3], prediction_type="RawFormulaVal")

array([[ 0.46872039,  0.02717889, -0.02942634, -0.23159877, -0.23487417],
       [ 0.16342239,  0.23999561,  0.08782089, -0.24248687, -0.24875202],
       [ 0.33317769,  0.15813378, -0.0085402 , -0.24050062, -0.24227064]])

In [0]:
model['oblivious_trees'] = trees[0:5]  # use only first 5 trees
json.dump(model, open("head_model.json", "w"))  # Save modified model
cls.load_model("head_model.json", "json")  # load model
cls.predict(X[0:3], prediction_type="RawFormulaVal")  # apply model

array([[ 0.27934996,  0.00999965, -0.04288901, -0.12313168, -0.12332892],
       [ 0.08037036,  0.13138144,  0.04135186, -0.124843  , -0.12826065],
       [ 0.19021373,  0.07576716, -0.01227257, -0.12547529, -0.12823304]])

In [0]:
model['oblivious_trees'] = trees[5:]  # drop first 5 trees
json.dump(model, open("tail_model.json", "w"))  # Save modified model
cls.load_model("tail_model.json", "json")  # load model
cls.predict(X[0:3], prediction_type="RawFormulaVal")  # apply model

array([[ 0.27934996,  0.00999965, -0.04288901, -0.12313168, -0.12332892],
       [ 0.08037036,  0.13138144,  0.04135186, -0.124843  , -0.12826065],
       [ 0.19021373,  0.07576716, -0.01227257, -0.12547529, -0.12823304]])